# Music Genre Classification 

In [15]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from tqdm import tqdm
import os

import librosa.display
import deeplake

from sklearn.model_selection import train_test_split

## Load dataset

In [12]:
ds = deeplake.load("hub://activeloop/gtzan-genre")

hub://activeloop/gtzan-genre loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/gtzan-genre
1000


## Augment data

In [13]:
add_noise = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.7),
])
pitch_shift = Compose([
    PitchShift(min_semitones=-4, max_semitones=12, p=0.5),
])

In [ ]:
#temporary list for the input data
data = []

#list to append all the labels
Y = []

base_path = '../input/gtzan-dataset-music-genre-classification/Data/genres_original'

#looping through all label directories
for label in tqdm(os.listdir(base_path)):
    file_path = base_path + '/' + label
    
    #looping through each file in the directory
    for pth in os.listdir(file_path):
        
        try:
            final_path = file_path + '/' + pth

            #loading original file
            audio, sr = librosa.load(final_path, duration = 28)
            
            #appending data to a list
            data.append(audio)
          

            #appending labels to the label list
            Y.append(label)
            
        except:
            print("Error in file", pth)
            pass
        
#converting list to a numpy array
X = np.stack(data)

In [ ]:

#split the data using the SkLearn library
audio_train, audio_test, y_train, y_test = train_test_split(\
     X, Y, test_size=0.20, random_state=6)

In [14]:

#setting melspec features
n_mels = 128
hop_length = 512
n_fft = 1024
sample_rate = 8000

#sample = ds

#audio_file =r'D:/1481637021654134785_sep.wav'
#audio_data, sr = librosa.load(audio_file, sr= 8000, mono=True)
#print(audio_data.shape)

#extract melspec features using librosa
#S = librosa.feature.melspectrogram(sample, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)

#convert it to DB scale
#S_DB = librosa.power_to_db(S, ref=np.max)

#display the spectrogram
#librosa.display.specshow(S_DB, sr=sample_rate, hop_length=hop_length, x_axis='time', y_axis='mel')
#plt.colorbar(format='%+2.0f dB')

C:\Users\ingvilcf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'D:/1481637021654134785_sep.wav'